<a href="https://colab.research.google.com/github/marinegyan/Makerevuytneri_erankyunacman_modelneri_naxagcman_algoritmeri_mshakum/blob/main/marching_cubes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import plotly.graph_objects as go
from skimage import measure


def f(x, y, z):
    return (x**2)/2 + (y**2)/3 + (z**2)/4 - 15

grid_size = 25
x = np.linspace(-10, 10, grid_size)
y = np.linspace(-10, 10, grid_size)
z = np.linspace(-10, 10, grid_size)
X, Y, Z = np.meshgrid(x, y, z)
F = f(X, Y, Z)

verts, faces, _, _ = measure.marching_cubes(F, level=0)
vx, vy, vz = verts[:,0], verts[:,1], verts[:,2]
scale_x = (x.max()-x.min())/grid_size
scale_y = (y.max()-y.min())/grid_size
scale_z = (z.max()-z.min())/grid_size
vx = vx*scale_x + x.min()
vy = vy*scale_y + y.min()
vz = vz*scale_z + z.min()


cube_edges = []
cube_size = (x[1] - x[0])

def make_cube_lines(x0, y0, z0, size):
    dx = [0, size]
    lines = []
    for sx in dx:
        for sy in dx:
            lines.append(([x0+sx, x0+sx], [y0+sy, y0+sy], [z0, z0+size]))
    for sx in dx:
        for sz in dx:
            lines.append(([x0+sx, x0+sx], [y0, y0+size], [z0+sz, z0+sz]))
    for sy in dx:
        for sz in dx:
            lines.append(([x0, x0+size], [y0+sy, y0+sy], [z0+sz, z0+sz]))
    return lines

for i in range(0, grid_size - 1, 2):
    for j in range(0, grid_size - 1, 2):
        for k in range(0, grid_size - 1, 2):
            vals = F[i:i+2, j:j+2, k:k+2]
            if np.min(vals) * np.max(vals) < 0 or abs(np.mean(vals)) < 25:
                cube_edges.extend(make_cube_lines(x[i], y[j], z[k], cube_size))

fig = go.Figure()

for lx, ly, lz in cube_edges:
    fig.add_trace(go.Scatter3d(
        x=lx, y=ly, z=lz,
        mode='lines',
        line=dict(color='rgba(100,100,100,0.35)', width=1),
        showlegend=False
    ))

fig.add_trace(go.Mesh3d(
    x=vx, y=vy, z=vz,
    i=faces[:,0], j=faces[:,1], k=faces[:,2],
    colorscale=[[0, "#5eb2a1"], [1, "#b4f8c8"]],  # gradient turquoise
    opacity=0.98,
    flatshading=True,
    lighting=dict(ambient=0.5, diffuse=1, specular=0.8, roughness=0.3, fresnel=0.2),
    lightposition=dict(x=80, y=100, z=120),
    name="Triangulated surface"
))

fig.add_trace(go.Isosurface(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(), value=F.flatten(),
    isomin=-0.1, isomax=0.1,
    surface_count=1,
    opacity=0.07,
    colorscale="Blues",
    caps=dict(x_show=False, y_show=False, z_show=False),
    showscale=False
))

fig.update_layout(
    title="3D Marching Cubes — Full Cell Scene (Thesis Visualization)",
    width=1100,
    height=900,
    paper_bgcolor='white',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        bgcolor='white',
        aspectmode='data',
        xaxis=dict(showbackground=False, gridcolor='lightgray'),
        yaxis=dict(showbackground=False, gridcolor='lightgray'),
        zaxis=dict(showbackground=False, gridcolor='lightgray'),
        camera=dict(
            eye=dict(x=1.6, y=1.6, z=1.1),
            center=dict(x=0, y=0, z=-0.1)
        )
    ),
    margin=dict(l=0, r=0, t=70, b=0)
)

fig.show()
